In [11]:
import pyaudio
import time
import queue


In [12]:
p = pyaudio.PyAudio()


CHUNK = 8192 #int(config.MIC_RATE / config.FPS))
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100 #config.MIC_RATE 
RECORD_SECONDS = 10


In [13]:
#view mics on system
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

Input Device id  0  -  HDA Intel PCH: ALC298 Analog (hw:0,0)
Input Device id  6  -  Scarlett 2i2 USB: Audio (hw:1,0)
Input Device id  8  -  sysdefault
Input Device id  10  -  pulse
Input Device id  11  -  asymed
Input Device id  12  -  pulse_playback
Input Device id  13  -  pulse_capture
Input Device id  14  -  default


In [14]:
#listen to mic

#choose input device:
input_device = 6

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK,
                input_device_index = input_device
               )


print("* recording")

frameL = []
frames = queue.Queue()

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frameL.append(data)
    frames.put(data)
    #time.sleep(0.1)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()


print(frames)
print(frames.get())

* recording
* done recording
b'\x00\x00\xbb\x04\xff\xff\xc8\x04\x00\x00\xd1\x04\x00\x00\xd2\x04\x00\x00\xdf\x04\x00\x00\xd9\x04\xff\xff\xda\x04\xff\xff\xe1\x04\x00\x00\xdb\x04\x00\x00\xdb\x04\x00\x00\xe6\x04\x00\x00\xcf\x04\x00\x00\xc7\x04\x00\x00\xc0\x04\x00\x00\xba\x04\xff\xff\xa6\x04\x00\x00\x98\x04\xff\xffx\x04\x00\x00x\x04\x00\x00\\\x04\x00\x00A\x04\x00\x00)\x04\xff\xff\x14\x04\xff\xff\xf2\x03\x00\x00\xd7\x03\x00\x00\xb9\x03\x00\x00\x94\x03\x00\x00v\x03\x00\x00\x10\x00\xff\xff\x06\x00\x01\x00\x02\x00\xff\xff\x0c\x00\x01\x00\t\x00\x01\x00\x06\x00\x00\x00\x0b\x00\x00\x00\xfe\xff\x00\x00\x04\x00\xfe\xff\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\xff\xff\x13\x00\x00\x00\x05\x00\x01\x00\xff\xff\xff\xff\xfc\xff\xff\xff\x08\x00\x00\x00\x07\x00\x00\x00\x05\x00\xfe\xff\x06\x00\xff\xff\x00\x00\xff\xff\n\x00\x00\x00\x02\x00\x01\x00\x00\x00\x01\x00\x04\x00\x00\x00\x02\x00\x00\x00\x04\x00\xff\xff\x02\x00\x00\x00\x0f\x00\xff\xff\x13\x00\x00\x00\xfd\xff\x00\x00\xf9\xff\x01\x00\x07\x00\xff\xff\r\x00\x

In [22]:
print(frames.get())

KeyboardInterrupt: 

In [11]:
print(frames.qsize())

5


In [12]:
import sys
print(sys.getsizeof(frames))
print(sys.getsizeof(frames)*frames.qsize())
print(sys.getsizeof(frames[0]))
print(sys.getsizeof(frames)*len(frames) * sys.getsizeof(frames[0]))

64
320


TypeError: 'Queue' object is not subscriptable

In [15]:
import wave

WAVE_OUTPUT_FILENAME = "recorded.wav"
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(p.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frameL))
waveFile.close()

In [ ]:

class AudioHandler(object):
    def __init__(self):
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024 * 2
        self.input_device = 6
        self.p = None
        self.stream = None

    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  input_device=self.input_device,
                                  stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK)

    def stop(self):
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        librosa.feature.mfcc(numpy_array)
        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            time.sleep(2.0)
            


audio = AudioHandler()
audio.start()     # open the the stream
audio.mainloop()  # main operations with librosa
audio.stop()
